In [ ]:
#1-import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import glob as gb
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
print("Packages imported...")

In [ ]:
#2-import dataset
data='C:/Users/wolfw/OneDrive/Desktop/Machine Learning/Project/Final Project/DataSet/asl_alphabet_train'

In [ ]:
for folder in  os.listdir(data) : 
    pathname= str( data +'/'+ folder)
    files = gb.glob(pathname)
    print(f'For training data , found {len(files)} in folder {folder}')
    print(pathname)

In [ ]:
for folder in  os.listdir(data) : 
    pathname= str( data +'/'+ folder)
    files = gb.glob(pathname)
    print(f'For training data , found {len(files)} in folder {folder}')
    print(pathname)

In [ ]:
#empty list to contain images itself called featues for the CNN model  
X_data = []
#empty list to contain actual value for each image
y_data = []
for folder in  os.listdir(data) : 
    pathname= str( data +'/'+ folder)
    files = gb.glob(os.path.join(pathname, '*'))
    for file in files: 
        image = cv2.imread(file)
        #using cv2.resize without determine interpolation make it preserve aspect ratio for each image 
        image_array = cv2.resize(image , (s,s))
        X_data.append(list(image_array))
        y_data.append(code[folder])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2,random_state=42,stratify=y_data)

In [ ]:
from keras.applications import VGG16
from keras.layers import Activation, Dense,Dropout
from tensorflow.keras import layers, models, Model, optimizers
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
vgg_model = VGG16(weights="imagenet", include_top=False, input_shape=(128, 128,3),classes=29)# include top to play on layers

for layer in vgg_model.layers[10:13]: ##control layer
    layer.trainable = False
   
for i, layer in enumerate(vgg_model.layers):
    print(i, layer.name, layer.trainable)


x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = Dense(256, activation='relu')(x)
x = Dense(29, activation='softmax')(x) # Softmax for multiclass1
transfer_model = Model(inputs=vgg_model.input, outputs=x)


lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint = ModelCheckpoint('vgg16_finetune.h15.keras', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)

def VGG16x(Train, labelTrain, Test, labelTest):
  learning_rate= 5e-5
  transfer_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=5e-5), metrics=["accuracy"])
  history = transfer_model.fit(Train, labelTrain, batch_size = 16, epochs=10, validation_data=(Test,labelTest), callbacks=[lr_reduce,checkpoint])
  transfer_model.save("VGG16.h5", include_optimizer=True)

In [ ]:
VGG16x(X_train,y_train,X_test,y_test)